# HomeWork

In [1]:
import tensorflow as tf
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
import numpy as np
from os import listdir,path
import random
import shutil

2023-06-13 15:12:14.324248: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Variables Declare

In [2]:
img_size=(128,128)
generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
trainDataPath = '/home/train'
testDataPath = '/home/test'
validationDataPath = '/home/validation'
batchSize = 32
classMode = 'categorical'
classficationLabels = listdir(trainDataPath)
arrX = []
arrY = []
datasetFolderName = path.join('')
trainData = None
testData = None
validData = None

# Load the data

In [3]:
def loadData(type):
    if type == 'train':
        
        trainData = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255,
            zoom_range=0.20,
            fill_mode="nearest"
        ).flow_from_directory(
            trainDataPath,
            target_size=img_size,
            batch_size=batchSize,
            class_mode= classMode
        )
        return trainData
    elif type == 'test':
        testData = generator.flow_from_directory(
            testDataPath,
            target_size=img_size,
            batch_size=batchSize,
            class_mode= classMode
        )
        return testData
    elif type == 'valid' :
        validData = generator.flow_from_directory(
            validationDataPath,
            target_size=img_size,
            batch_size=batchSize,
            class_mode= None,
            shuffle=False
        )
        return validData



# Define the Model

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)), # Layer 1
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'), # Layer 2
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'), # Layer 3
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(), # Full c. layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

2023-06-13 15:12:20.143534: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1c:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-13 15:12:20.151380: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1c:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-13 15:12:20.152500: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1c:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-13 15:12:20.155400: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1c:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-13 15:12:20.155956: I tensorflow/compile

# Compile the model

In [5]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Kfold Functions Define

In [6]:
def move(src,des,spiltRate):
    srcFiles = listdir(src)
    if(len(srcFiles) != 0):
        numberOfTransfer = int(len(srcFiles)*spiltRate)
        transferIndex = random.sample(range(0,len(srcFiles)),numberOfTransfer)
        for index in transferIndex :
            shutil.move(path.join(src,str(srcFiles[index])),
                        path.join(des,str(srcFiles[index])))
    else:
        print("No file move")

def moveClass(src=trainDataPath,des=validationDataPath,spiltRate=0.2):
    for label in classficationLabels:
        move(path.join(src,label),
             path.join(des,label),
             spiltRate
            )
        
def fillArr():
    X=[]
    Y=[]
    for folder in classficationLabels:
        srcFiles = listdir(path.join(trainDataPath,folder))
        yLabel = 0
        for i in range(len(classficationLabels)):
            if(folder == classficationLabels[i]):
                yLabel = i
        for imageName in srcFiles:
            X.append(imageName)
            Y.append(yLabel)
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X,Y

def meansure(anwser,predict):
    accuracy = accuracy_score(anwser,predict)
    precision = precision_score(anwser,predict,average='weighted')
    f1 = f1_score(anwser,predict,average='weighted')
    print("Accuracy : %d\nPrecision : %d\nf1Score : %d\n" %(accuracy,precision,f1))
    confusionMatrix = confusion_matrix(anwser,predict)
    print("Confusion_Matrix : \n{}".format(confusionMatrix))
    return accuracy,precision,f1


# Kfold

In [7]:
moveClass(validationDataPath,trainDataPath,1.0)
arrX,arrY = fillArr()
fold = StratifiedKFold(n_splits=4,shuffle=True)
fold.get_n_splits(arrX,arrY)
foldIndex = 0
results = []
for trainIndex,labelIndex in fold.split(arrX,arrY):
    moveClass()
    foldIndex += 1
    print("Fold :",foldIndex)
    xTrain,xValue = arrX[trainIndex],arrX[labelIndex]
    yTrain,yValue = arrY[trainIndex],arrY[labelIndex]
    trainData = loadData('train')
    validData = loadData('valid')
    result = model.fit(
        trainData,
        epochs=25,
        verbose=1,
        validation_data=validData,
        steps_per_epoch=trainData.samples/trainData.batch_size,
        validation_steps=validData.samples/validData.batch_size
        )
    moveClass(validationDataPath,trainDataPath,1.0)
    results.append(result.history['accuracy'][len(result.history['accuracy'])-1])



No file move
No file move
No file move
No file move
No file move
No file move
No file move
No file move
No file move
No file move
Fold : 1
Found 800 images belonging to 10 classes.
Found 200 images belonging to 10 classes.
Epoch 1/25


2023-06-13 15:12:29.112414: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-13 15:12:30.899351: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-06-13 15:12:33.052399: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-06-13 15:12:34.624362: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fd73400db40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-13 15:12:34.624496: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor de

25/25 [==============================] - ETA: 0s - loss: 2.7415 - accuracy: 0.1075

2023-06-13 15:12:43.751299: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


25/25 [==============================] - 16s 322ms/step - loss: 2.7415 - accuracy: 0.1075 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/25
25/25 [==============================] - 9s 361ms/step - loss: 2.3029 - accuracy: 0.0875 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/25
25/25 [==============================] - 9s 347ms/step - loss: 2.3028 - accuracy: 0.1025 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/25
25/25 [==============================] - 8s 321ms/step - loss: 2.2932 - accuracy: 0.1450 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/25
25/25 [==============================] - 8s 330ms/step - loss: 2.1126 - accuracy: 0.2313 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/25
25/25 [==============================] - 7s 296ms/step - loss: 1.8066 - accuracy: 0.3762 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/25
25/25 [==============================] - 7s 286ms/step - loss: 1.4979 - accuracy: 0.4900 - val_loss

...

25/25 [==============================] - 8s 315ms/step - loss: 0.0907 - accuracy: 0.9762 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/25
25/25 [==============================] - 8s 304ms/step - loss: 0.1058 - accuracy: 0.9712 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/25
25/25 [==============================] - 8s 308ms/step - loss: 0.0539 - accuracy: 0.9812 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/25
25/25 [==============================] - 8s 302ms/step - loss: 0.0484 - accuracy: 0.9850 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/25
25/25 [==============================] - 9s 351ms/step - loss: 0.0518 - accuracy: 0.9850 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/25
25/25 [==============================] - 8s 308ms/step - loss: 0.0493 - accuracy: 0.9825 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/25
25/25 [==============================] - 7s 293ms/step - loss: 0.0583 - accuracy: 0.9825 - val_loss:

2023-06-13 15:22:34.190149: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


25/25 [==============================] - ETA: 0s - loss: 0.0680 - accuracy: 0.9800

2023-06-13 15:22:41.597295: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


25/25 [==============================] - 8s 324ms/step - loss: 0.0680 - accuracy: 0.9800 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/25
25/25 [==============================] - 8s 312ms/step - loss: 0.0733 - accuracy: 0.9875 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/25
25/25 [==============================] - 8s 317ms/step - loss: 0.0683 - accuracy: 0.9787 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/25
25/25 [==============================] - 8s 319ms/step - loss: 0.0400 - accuracy: 0.9875 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/25
25/25 [==============================] - 8s 315ms/step - loss: 0.0371 - accuracy: 0.9925 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/25
25/25 [==============================] - 8s 311ms/step - loss: 0.0222 - accuracy: 0.9950 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/25
25/25 [==============================] - 8s 310ms/step - loss: 0.0353 - accuracy: 0.9887 - val_loss:

In [8]:
index = 0
for value in results:
    index+=1
    print("Fold %d : %.3f" %(index,value))

Fold 1 : 0.961
Fold 2 : 0.990
Fold 3 : 0.996
Fold 4 : 0.991


In [11]:
loss,acc = model.evaluate(loadData("test"))
print("Test Accuracy : %3.3f" %(acc))

Found 500 images belonging to 10 classes.


2023-06-13 15:28:37.274245: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


16/16 [==============================] - 4s 238ms/step - loss: 0.6849 - accuracy: 0.9020
Test Accuracy : 0.902


In [9]:
# model.save(filepath='/home/result.h5')
